In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers



In [2]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ["num_pregnancies", "glucose_concentration", "blood_pressure", "skin_thickness",
           "insulin", "bmi", "diabetes_pedigree", "age", "outcome"]

In [3]:
data = pd.read_csv(url, names=columns)
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data.drop(columns=["outcome"],axis=1))

In [4]:
#tworzenie generatora
def build_generator(latent_dim):
  model = tf.keras.Sequential([
      layers.Dense(128,activation="relu",input_shape=latent_dim),
      layers.Dense(256,activation="relu"),
      layers.Dense(512,activation="relu"),
      layers.Dense(data_normalized.shape[1],activation="sigmoid")
  ])
  return model

In [5]:
#tworzenie dyskryminatora
def build_discriminator():
  model = tf.keras.Sequential([
      layers.Dense(512,activation="relu",input_shape=(data_normalized.shape[1],)),
      layers.Dense(256,activation="relu"),
      layers.Dense(128,activation="relu"),
      layers.Dense(1,activation="sigmoid")
  ])
  return model

In [11]:
#tworzenie sieci GAN
latent_dim = 10
generator = build_generator((latent_dim,))
discriminator = build_discriminator()

#kompilacja dyskryminatora
discriminator.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

#tworzenie GAN
discriminator.trainable = False
gan_input = layers.Input(shape=(latent_dim,))
generated_data = generator(gan_input)
gan_output = discriminator(generated_data)
gan = tf.keras.models.Model(gan_input,gan_output)
gan.compile(loss="binary_crossentropy",optimizer="adam")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
#trening modelu GAN
def train_gan(generator,discriminator,epochs=100,batch_size=128):
  half_batch = batch_size // 2
  for epoch in range(epochs):
    idx = np.random.randint(0,data_normalized.shape[0],half_batch)
    real_data = data_normalized[idx]

    noise = np.random.normal(0,1,(half_batch,latent_dim))
    generator_data = generator.predict(noise)

    discriminator_loss_real = discriminator.train_on_batch(real_data,np.ones((half_batch,1)))
    discriminator_loss_fake = discriminator.train_on_batch(generator_data,np.zeros((half_batch,1)))
    discriminator_loss = 0.5 * np.add(discriminator_loss_real,discriminator_loss_fake)

    noise = np.random.normal(0,1,(batch_size,latent_dim))
    generator_loss = gan.train_on_batch(noise,np.ones((batch_size,1)))

    if epoch%100==0:
      print(f"Epoch {epoch}/{epochs} | Discriminator Loss: {discriminator_loss[0]} | Generator Loss: {generator_loss}")

In [ ]:
train_gan(generator,discriminator,epochs=1000,batch_size=128)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step  


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0/1000 | Discriminator Loss: 0.7022534608840942 | Generator Loss: [array(0.691025, dtype=float32), array(0.691025, dtype=float32), array(0.5, dtype=float32)]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━